In [1]:
import sys
import os
import json
import logging
from google.cloud import secretmanager

#hegemon dependencis
from hegemon.explainability.visualizer import HeatmapGenerator
from hegemon.config.settings import get_settings

/opt/conda/envs/hegemon_env/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py:63: UserWarning: `validate_explainability_config` overrides an existing Pydantic `@model_validator` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')


In [2]:
PROJECT_ID = "dark-data-discovery"
LOCATION = "us-central1"

print(f"📍 GCP Project: {PROJECT_ID}")
print(f"📍 Location: {LOCATION}")

📍 GCP Project: dark-data-discovery
📍 Location: us-central1


In [3]:
def get_secret(project_id, secret_id, version_id="latest"):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")

try:
    key = get_secret(PROJECT_ID, "ANTHROPIC_API_KEY")
    print(f"✅ ANTHROPIC_API_KEY: {'*' * 10}{key[-4:]}")
except Exception as e:
    print(f"❌ Failed to get ANTHROPIC_API_KEY: {e}")

✅ ANTHROPIC_API_KEY: **********uwAA


In [4]:
# Komórka 1: Setup
# ENABLE EXPLAINABILITY (przez os.environ)
os.environ["HEGEMON_EXPLAINABILITY_ENABLED"] = "true"
os.environ["HEGEMON_EXPLAINABILITY_SEMANTIC_FINGERPRINT"] = "true"
os.environ["HEGEMON_EXPLAINABILITY_CLASSIFIER_MODEL"] = "gemini-2.5-flash"  # Vertex AI model

# Setup logging dla Jupyter
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True
)
logger = logging.getLogger(__name__)

print("✅ Environment configured for Vertex AI")
print(f"   Explainability enabled: {os.environ.get('HEGEMON_EXPLAINABILITY_ENABLED')}")
print(f"   Classifier model: {os.environ.get('HEGEMON_EXPLAINABILITY_CLASSIFIER_MODEL')}")

✅ Environment configured for Vertex AI
   Explainability enabled: true
   Classifier model: gemini-2.5-flash


In [5]:
# Komórka 2: Import Hegemon modules
from hegemon.explainability.visualizer import HeatmapGenerator


# Verify settings
settings = get_settings()
print("Explainability Layers:")
print(f"  Layer 6 (Semantic): {settings.explainability_semantic_fingerprint}")
print(f"  Layer 2 (Epistemic): {settings.explainability_epistemic_uncertainty}")


print("=" * 80)
print("✅ STEP 2: Settings loaded and verified")
print("=" * 80)
print(f"GCP Project ID: {settings.gcp_project_id}")
print(f"GCP Location: {settings.gcp_location}")
print()
print("🔍 EXPLAINABILITY SETTINGS:")
print(f"   enabled: {settings.explainability_enabled}")
print(f"   semantic_fingerprint: {settings.explainability_semantic_fingerprint}")
print(f"   classifier_model: {settings.explainability_classifier_model}")
print(f"   cache_size: {settings.explainability_cache_size}")
print("=" * 80)

# CRITICAL CHECK
if not settings.explainability_enabled:
    print("\n❌ ERROR: Explainability is DISABLED!")
    print("   This means environment variable was set AFTER settings were cached.")
    print("   SOLUTION: Restart kernel and run cells in correct order.")
else:
    print("\n✅ SUCCESS: Explainability is ENABLED!")
    print("   Proceed to next cells.")

print()

2025-10-12 23:47:49,318 - hegemon.config.settings - WARNING - Explainability enabled but no Google API key found. Classifier will fail.
2025-10-12 23:47:49,319 - hegemon.config.settings - INFO - ✅ Sceptyk will use Vertex AI (Application Default Credentials)
2025-10-12 23:47:49,321 - hegemon.config.settings - INFO - ✅ All required authentication validated
2025-10-12 23:47:49,321 - hegemon.config.settings - INFO - ✅ HEGEMON Settings initialized (GCP Project: dark-data-discovery)


Explainability Layers:
  Layer 6 (Semantic): True
  Layer 2 (Epistemic): True
✅ STEP 2: Settings loaded and verified
GCP Project ID: dark-data-discovery
GCP Location: us-central1

🔍 EXPLAINABILITY SETTINGS:
   enabled: True
   semantic_fingerprint: True
   classifier_model: gemini-2.0-flash
   cache_size: 1000

✅ SUCCESS: Explainability is ENABLED!
   Proceed to next cells.



In [6]:
from hegemon.graph_hitl_v2 import create_hegemon_graph_hitl_v2
from hegemon.hitl import HumanFeedback

# Create graph
graph = create_hegemon_graph_hitl_v2()

# Test with observer mode (no feedback)
initial_state = {
    "mission": "Design a simple web API",
    "intervention_mode": "observer",
    "contributions": [],
    "cycle_count": 1,
    "current_consensus_score": 0.0,
    "final_plan": None,
    "current_checkpoint": None,
    "human_feedback_history": [],
    "paused_at": None,
    "revision_count_per_checkpoint": {},
    "checkpoint_snapshots": {},
}

result = graph.invoke(initial_state, config={"recursion_limit": 100})

print(f"✅ Completed in {result['cycle_count']} cycles")
print(f"   Consensus: {result['current_consensus_score']:.2f}")

2025-10-12 23:47:49,350 - hegemon.graph_hitl_v2 - INFO - 🏗️ Building HEGEMON graph with Phase 2.2 feedback-aware agents...
2025-10-12 23:47:49,378 - hegemon.graph_hitl_v2 - INFO - ✅ HEGEMON graph Phase 2.2 compiled successfully!
2025-10-12 23:47:49,379 - hegemon.graph_hitl_v2 - INFO -    Enhanced: Feedback-aware agents with effectiveness tracking
2025-10-12 23:47:49,379 - hegemon.graph_hitl_v2 - INFO -    Checkpoints: post_thesis, post_evaluation, pre_synthesis
2025-10-12 23:47:49,383 - hegemon.agents_hitl - INFO - 🔥 KATALIZATOR (anthropic/claude-sonnet-4-5-20250929): Generating Thesis (Cycle 1) [NO FEEDBACK]
2025-10-12 23:47:49,391 - hegemon.agents_hitl - INFO - ✅ KATALIZATOR: Thesis generated (2278 chars)
2025-10-12 23:47:49,392 - hegemon.explainability.collector - WARNING - Failed to get explainability collector: 'HegemonSettings' object has no attribute 'explainability'
2025-10-12 23:47:49,394 - hegemon.hitl.checkpoints - INFO - 🔭 Observer mode active - skipping post_thesis checkpo

✅ Completed in 3 cycles
   Consensus: 0.73


In [9]:
from hegemon.hitl import compute_feedback_effectiveness, HumanFeedback

feedback = HumanFeedback(
    checkpoint="post_thesis_cycle_1",
    decision="revise",
    guidance="Add cost estimates and timeline",
)

original_output = "We should use microservices."
revised_output = "We should use microservices with estimated cost of $50K over 6 months."

# Compute effectiveness
result = compute_feedback_effectiveness(original_output, revised_output, feedback)

print(f"Overall: {result['overall']:.2f} ({result['interpretation']})")
print(f"Structural: {result['structural']:.2f}")
print(f"Keyword Match: {result['keyword_match']:.2f}")

2025-10-12 23:50:32,514 - hegemon.hitl.effectiveness - INFO - Effectiveness score: 0.45 (Fair) - Structural: 0.61, Keywords: 0.33


Overall: 0.45 (Fair)
Structural: 0.61
Keyword Match: 0.33


In [14]:
from hegemon.hitl import detect_feedback_contradictions, generate_contradiction_report

# Analyze feedback history - BEZPIECZNY sposób
feedback_history = result.get("human_feedback_history", [])

if not feedback_history:
    print("ℹ️ No feedback history (observer mode or no interventions)")
else:
    contradictions = detect_feedback_contradictions(feedback_history)
    
    if contradictions:
        report = generate_contradiction_report(contradictions)
        print(report)
        
        # Save report
        with open("contradiction_report.md", "w") as f:
            f.write(report)
        
        print(f"\n💾 Report saved to: contradiction_report.md")
    else:
        print("✅ No contradictions detected")

ℹ️ No feedback history (observer mode or no interventions)


In [16]:
from hegemon.hitl import build_agent_prompt_with_feedback

# Użyj RESULT z debaty (nie 'state')
enhanced_system, enhanced_user = build_agent_prompt_with_feedback(
    state=result,  # ← Użyj wyniku debaty
    agent_id="Katalizator",
    base_system_prompt="You are Katalizator agent.",
    base_user_prompt=f"Mission: {result['mission']}",  # ← Dynamicznie pobierz mission
    include_debate_context=True,
)

print("Enhanced System Prompt:")
print(enhanced_system)
print("\n" + "="*80 + "\n")
print("Enhanced User Prompt:")
print(enhanced_user)

KeyError: 'mission'

In [17]:
from hegemon.hitl import build_agent_prompt_with_feedback, HumanFeedback

# Stwórz testowy state Z feedbackiem
test_state = {
    "mission": "Design microservices platform",
    "contributions": [],
    "cycle_count": 1,
    "current_consensus_score": 0.0,
    "final_plan": None,
    "intervention_mode": "reviewer",
    "current_checkpoint": None,
    "human_feedback_history": [
        HumanFeedback(
            checkpoint="post_thesis_cycle_1",
            decision="revise",
            guidance="Add detailed cost breakdown and ROI analysis",
            priority_claims=["Cost estimation", "ROI timeline"],
            flagged_concerns=["Hidden operational costs", "Scalability costs"],
        ).model_dump()
    ],
    "paused_at": None,
    "revision_count_per_checkpoint": {},
    "checkpoint_snapshots": {},
}

# Teraz użyj tego state
enhanced_system, enhanced_user = build_agent_prompt_with_feedback(
    state=test_state,
    agent_id="Katalizator",
    base_system_prompt="You are Katalizator agent.",
    base_user_prompt=f"Mission: {test_state['mission']}",
    include_debate_context=False,
)

print("=" * 80)
print("ENHANCED SYSTEM PROMPT (with feedback):")
print("=" * 80)
print(enhanced_system)

print("\n" + "=" * 80)
print("ENHANCED USER PROMPT:")
print("=" * 80)
print(enhanced_user)

ENHANCED SYSTEM PROMPT (with feedback):
You are Katalizator agent.

╔══════════════════════════════════════════════════════════════╗
║ HUMAN FEEDBACK (1 revision requested)                              ║
╚══════════════════════════════════════════════════════════════╝

The human reviewed your previous output and requested changes:

🔄 **Decision: REVISE**

📝 **Guidance:**
"Add detailed cost breakdown and ROI analysis"

🎯 **Priority Items to Emphasize:**
• Cost estimation
• ROI timeline

⚠️ **Concerns to Address:**
• Hidden operational costs
• Scalability costs

**ACTION REQUIRED:** Please incorporate this feedback in your revised output.


ENHANCED USER PROMPT:
Mission: Design microservices platform
